# How to spinfoam amplitudes
## Computing the EPRL vertex amplitudes

In this notebook we show how to compute the EPRL vertices. 

We employ the [sl2cfoam-next](https://github.com/qg-cpt-marseille/sl2cfoam-next) library.
See also [paper describing the library](https://arxiv.org/abs/2107.13952).

In order to compile the cells below you first need to install the sl2cfoam-next.

## Single vertex

We first focus on the the computation of a single amplitude

In [3]:
# required pkgs  
using HalfIntegers
using SL2Cfoam

# Barbero-Immirzi parameter
γ = 1.2

# initializing sl2cfoam-next   
sl2c_data_folder = "/home/frisus95/Scrivania/sl2cfoam_next/data_sl2cfoam"
sl2c_configuration = SL2Cfoam.Config(VerbosityOff, VeryHighAccuracy, 100, 0)
sl2c_result_return = (ret = true, store = false, store_batches = false)
SL2Cfoam.cinit(sl2c_data_folder, γ, sl2c_configuration) 

# homogeneous cut-off 
Δl = 0;

### Full vertex

We compute the EPRL vertex amplitude for each possible value of the boundary intertwiners

In [30]:
# set the 10 vertex boundary spins
j245, j125, j124, j145, j235, j234, j345, j123, j135, j134 = [1 for _ = 1:10]

spins = [j245, j125, j124, j145, j235, j234, j345, j123, j135, j134]

@time v = vertex_compute(spins, Δl; result = sl2c_result_return);

# v.a is the 5-dimensional array with the data
vertex = v.a

  0.011231 seconds (7 allocations: 400 bytes)


3×3×3×3×3 Array{Float64, 5}:
[:, :, 1, 1, 1] =
  6.68474e-11  -8.72166e-11  1.27075e-10
 -9.58955e-11   7.55318e-11  1.01156e-10
  1.3407e-10    9.75111e-11  3.53041e-11

[:, :, 2, 1, 1] =
 -8.72166e-11   6.83145e-11  9.16939e-11
  1.437e-10    -5.91621e-11  8.27024e-11
 -1.89318e-10  -7.63779e-11  1.79525e-11

[:, :, 3, 1, 1] =
  1.27075e-10   9.16939e-11  3.35534e-11
 -1.97308e-10  -7.94092e-11  1.88648e-11
  2.66493e-10  -1.02517e-10  1.53984e-11

[:, :, 1, 2, 1] =
 -9.58955e-11   1.437e-10    -1.97308e-10
  1.36483e-10  -1.24448e-10  -1.5763e-10
 -1.9149e-10   -1.60661e-10  -5.43777e-11

[:, :, 2, 2, 1] =
  7.55318e-11  -5.91621e-11  -7.94092e-11
 -1.24448e-10   5.12359e-11  -7.16223e-11
  1.63955e-10   6.61452e-11  -1.55473e-11

[:, :, 3, 2, 1] =
  1.01156e-10   8.27024e-11  1.88648e-11
 -1.5763e-10   -7.16223e-11  8.47642e-12
  2.12577e-10  -9.2464e-11   1.03074e-11

[:, :, 1, 3, 1] =
  1.3407e-10   -1.89318e-10  2.66493e-10
 -1.9149e-10    1.63955e-10  2.12577e-10
  2.68244e-10 

### Range for boundary intertwiners

We can also compute the EPRL vertex amplitude for a restricted range of boundary intertwiners

In [5]:
# initialize each boundary spins with a random integer value between 1 and 3
j245, j125, j124, j145, j235, j234, j345, j123, j135, j134 = [rand(1:3) for _ = 1:10]

spins = [j245, j125, j124, j145, j235, j234, j345, j123, j135, j134]

@show spins

# determine the range of boundary intertwiners
r1245, _ = intertwiner_range(j245, j125, j124, j145)
r2345, _ = intertwiner_range(j235, j234, j345, j245) 
r1235, _ = intertwiner_range(j123, j135, j125, j235)
r1234, _ = intertwiner_range(j134, j124, j234, j123)
r1345, _ = intertwiner_range(j145, j345, j135, j134)
    
# intertwiners fixed to their minimum value
i1245 =  r1245[1]  
i2345 =  r2345[1]   
i1345 =  r1345[1]   

# restricted range of boundary intertwiners to compute
vertex_intertw_ranges = ((i1245,i1245), (i2345, i2345), r1235, r1234, (i1345,i1345))

@show vertex_intertw_ranges

@time v = vertex_compute(spins, Δl, vertex_intertw_ranges; result = sl2c_result_return);

vertex = v.a

spins = [2, 3, 1, 1, 3, 2, 3, 2, 1, 1]
vertex_intertw_ranges = ((1, 1), (1, 1), (1, 3), (0, 2), (2, 2))
  0.040771 seconds (56.73 k allocations: 2.953 MiB, 72.39% compilation time)


1×3×3×1×1 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 -8.42174e-15  2.01586e-13  -1.41033e-13

[:, :, 2, 1, 1] =
 5.91758e-14  -2.92793e-13  9.90976e-13

[:, :, 3, 1, 1] =
 -1.24593e-13  3.26532e-13  -2.08647e-12

## Vertices for the Δ4 amplitude

We now compute and store all EPRL vertices required for the computation of the Δ4 amplitude.

The vertices are computed for increasing values of the cut-off Δl.

For simplicity we set all boundary spins equal to j, computing the full vertices

In [2]:
# required pkgs
using JLD2   
using DelimitedFiles
using Dates

# current folder
root_dir = pwd()

# print with flush
function log(x...)
println(now(), " ", join(x, " ")...)
flush(stdout)
end

# maximal cut-off  
Δl_max = 15;

In [42]:
# value of all boundary spins 
j = 1.0

for Δl = 0:Δl_max
    
   # folder with vertex amplitudes 
   vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                 
   mkpath(vertex_path)
    
   # if all boundary spins are integers and equal to j, bulk spin has range [0, 3j]   
   j_bulk_min, j_bulk_max = 0, 3j
    
   for j_bulk = j_bulk_min:j_bulk_max
    
      v = vertex_compute([j, j, j, j, j, j, j, j_bulk, j, j], Δl; result = sl2c_result_return);        
      log("computed full vertex tensor for Immirzi=$(γ) j_boundary=$(j) j_bulk=$(j_bulk) Dl=$(Δl)")
                
      vertex = v.a;        
      @save "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2" vertex         
               
   end # bulk spin cycle
      
end # Δl cycle

2022-01-27T10:28:24.969 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=0.0 Dl=0
2022-01-27T10:28:27.603 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=1.0 Dl=0
2022-01-27T10:28:27.630 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=2.0 Dl=0
2022-01-27T10:28:27.651 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=3.0 Dl=0
2022-01-27T10:28:27.654 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=0.0 Dl=1
2022-01-27T10:28:27.656 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=1.0 Dl=1
2022-01-27T10:28:27.658 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=2.0 Dl=1
2022-01-27T10:28:27.667 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=3.0 Dl=1
2022-01-27T10:28:27.679 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=0.0 Dl=2
2022-01-27T10:28:27.681 computed full vertex tensor for Immirzi=1.2 j_boundary=1.0 j_bulk=1.0 Dl=2
2022-01-27

For convenience, we write a function that computes all the required full vertices for given lists of boundary parameters in the symmetric configuration

In [4]:
function initializing_library(Immirzi)
   
folder = "/home/frisus95/Scrivania/sl2cfoam_next/data_sl2cfoam";
conf = SL2Cfoam.Config(VerbosityOff, VeryHighAccuracy, 100, 0);
SL2Cfoam.cinit(folder, Immirzi, conf);
    
end    

function Δ4_vertices(Immirzi_list, j_boundary_list, Δl_max)

for γ in Immirzi_list
    
   initializing_library(γ)    

   for j in j_boundary_list   
    
       for Δl = 0:Δl_max
        
             vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                 
             mkpath(vertex_path)
                
             if (iszero(j%1)) j_bulk_min = 0 else j_bulk_min = 0.5 end       
             j_bulk_max = 3j
    
             for j_bulk = j_bulk_min:j_bulk_max
    
                v = vertex_compute([j, j, j, j, j, j, j, j_bulk, j, j], Δl; result = sl2c_result_return);        
                log("computed full vertex tensor for Immirzi=$(γ) j_boundary=$(j) j_bulk=$(j_bulk) Δl=$(Δl)")
                
                vertex = v.a;        
                @save "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2" vertex         
               
             end # bulk spin cycle
      
        end # cut-off cycle
            
    end # j_boundary cycle
    
end  # Immirzi cycle
    
end

Δ4_vertices (generic function with 1 method)

In [5]:
# list of Immirzi parameters 
Immirzi_list = [0.1, 1.2]

# list of boundary spins (symmetric configurations)
j_boundary_list = [0.5,1]

# maximal cut-off
Δl_max = 15;

In [6]:
Δ4_vertices(Immirzi_list, j_boundary_list, Δl_max)

2022-01-27T10:43:43.188 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Δl=0
2022-01-27T10:43:44.651 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Δl=0
2022-01-27T10:43:44.664 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Δl=1
2022-01-27T10:43:44.674 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Δl=1
2022-01-27T10:43:44.677 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Δl=2
2022-01-27T10:43:44.686 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Δl=2
2022-01-27T10:43:44.699 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Δl=3
2022-01-27T10:43:44.707 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Δl=3
2022-01-27T10:43:44.720 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Δl=4
2022-01-27T10:43:44.731 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Δl=4
2022-01-27